In [1]:
# William Nourse
# Feb 18 2021

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from ipywidgets import interact_manual
%matplotlib widget

In [2]:
S = 0.05
R = 20
tauHmax = 300 # ms

#delEna = Ena - Er
Utest = np.arange(-R,3*R,0.1)
mInf = lambda U: 1/(1+np.exp(S*(R-U)))
hInf = lambda U: 1/(1+0.5*np.exp(S*U))
tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

plt.figure(figsize=(30,15))
plt.subplot(1,2,1)
plt.plot(Utest,mInf(Utest),label='$m_{\infty}(U)$')
plt.plot(Utest,hInf(Utest),label='$h_{\infty}(U)$')
plt.xlabel('U (mV)')
plt.title('Gating Variables')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Utest,tauh(Utest))
plt.xlabel('U (mV)')
plt.title('Gating Time Constant')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Gating Time Constant')

In [3]:
def singleNeuronPlot():
    # Parameters
    C = 5 # nF
    tauHmax = 300 # ms
    Gm = 1 # uS
    Ena = 50 # mV

    I = 0
    tStart = 1000
    tEnd = 4000
    Er = -60
    k = -1
    tTail = 2500

    
    delta = -0.01
    dt = 1
    tMax = 5000
    R = 20
    t = np.arange(0,tMax,dt)
    numSteps = np.size(t)

    Iapp = np.zeros(numSteps)
    for i in range(numSteps):
        if (t[i]>= tStart & t[i]<=tEnd):
            Iapp[i] = I

    Ipert = np.zeros(numSteps)
    Ipert[1] = 1

    # Design the synapse
    Ein = -100
    delEsyn = Ein - R

    # persistent sodium channel
    S = 0.05
    delEna = Ena - Er
    Utest = np.arange(-R,3*R,0.1)
    mInf = lambda U: 1/(1+np.exp(S*(R-U)))
    hInf = lambda U: 1/(1+0.5*np.exp(S*U))
    tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

    # Channel conductance
    Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

    # h* and m*
    Ustar = R
    mStar = mInf(Ustar)
    hStar = hInf(Ustar)

    Unull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))

    # Compute simulation
    U1 = np.zeros(numSteps)
    h1 = np.zeros(numSteps)
    h1[0] = hInf(U1[0])

    g = lambda U: min(max(U/R,0),1)*gSyn


    for i in range(1,numSteps):
        U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1]);
        h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1]);

    grid = plt.GridSpec(3,2)
    plt.figure(figsize=(30,15))
    plt.subplot(grid[0,0])
    plt.plot(t,Iapp+Ipert,lw=5)
    plt.ylabel('$I_{app}$ (nA)')

    plt.subplot(grid[1,0])
    plt.plot(t,U1,lw=5)
    plt.ylabel('U (mV)')

    plt.subplot(grid[2,0])
    plt.plot(t,h1,lw=5)
    plt.ylabel('h')

    plt.subplot(grid[:,1])
    plt.axvline(x=0,linestyle=':',color='k',label='Synaptic Threshold',lw=5)
    plt.xlim([-R,3*R])
    plt.ylim([0,1])
    plt.plot(Utest,hInf(Utest),':',label='h Nullcline',lw=5)
    plt.plot(Utest,Unull(Utest,Iapp[0]),label='U Nullcline',lw=5)
    plt.plot(U1,h1,label='Neuron',lw=5)
    plt.xlabel('U (mV)')
    plt.ylabel('h (mV)')
    plt.legend()
    
singleNeuronPlot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def doubleNeuronPlot(delta=-0.01,S=0.05,lim=[-R,3*R,0,1]):
    # Parameters
    C = 5 # nF
    tauHmax = 300 # ms
    Gm = 1 # uS
    Ena = 50 # mV

    I = 0
    tStart = 1000
    tEnd = 4000
    Er = -60
    k = -1
    tTail = 2500

    dt = 1
    tMax = 5000
    R = 20
    t = np.arange(0,tMax,dt)
    numSteps = np.size(t)

    Iapp = np.zeros(numSteps)
    for i in range(numSteps):
        if (t[i]>= tStart & t[i]<=tEnd):
            Iapp[i] = I

    Ipert = np.zeros(numSteps)
    Ipert[1] = 1

    # Design the synapse
    Ein = -100
    delEsyn = Ein - R

    # persistent sodium channel
    delEna = Ena - Er
    Utest = np.arange(-R,3*R,0.1)
    mInf = lambda U: 1/(1+np.exp(S*(R-U)))
    hInf = lambda U: 1/(1+0.5*np.exp(S*U))
    tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

    # Channel conductance
    Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

    # h* and m*
    Ustar = R
    mStar = mInf(Ustar)
    hStar = hInf(Ustar)

    # compute jacobian matrix and its eigenvalues
    dU = 1e-3
    dm_dU = lambda U: (mInf(U+dU)-mInf(U))/dU
    dh_dU = lambda U: (hInf(U+dU)-hInf(U))/dU
    dtauh_dU = lambda U: (tauh(U+dU)-tauh(U))/dU

    dUdot_dU = 1/C*(-1+dm_dU(Ustar)*Gna*hStar*(delEna - Ustar) - Gna*mStar*hStar)
    dUdot_dh = 1/C*(Gna*mStar*(delEna - Ustar))
    dhdot_dU = dh_dU(Ustar)/tauh(Ustar)
    dhdot_dh = -1/tauh(Ustar)

    J = np.array([[dUdot_dU, dUdot_dh],
                   [dhdot_dU, dhdot_dh]])

    eigVals,_ = np.linalg.eig(J)
    if np.imag(J).any():
        print('Warning: Complex Eigenvalues!')

    dUdt = lambda U: -Gm*U + Gna*mInf(U)*hInf(U)*(delEna-U)

    gSyn = (-delta - delta*Gna*mInf(delta)*hInf(delta) + Gna*mInf(delta)*hInf(delta)*delEna)/(delta - delEsyn)
   
    unUnull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))
    inUnull = lambda U,I: (Gm*U - gSyn*(delEsyn - U) - I)/(Gna*mInf(U)*(delEna - U))

    # Compute simulation
    U1 = np.zeros(numSteps)
    U2 = np.zeros(numSteps)
    h1 = np.zeros(numSteps)
    h1[0] = hInf(U1[0])
    h2 = np.zeros(numSteps)
    h2[0] = hInf(U2[0])

    g = lambda U: min(max(U/R,0),1)*gSyn

    Gsyn1 = np.zeros(numSteps)
    Gsyn1[0] = g(U1[0])
    Gsyn2 = np.zeros(numSteps)
    Gsyn2[0] = g(U2[0])

    for i in range(1,numSteps):
        U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gsyn2[i-1]*(delEsyn-U1[i-1]) + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1]);
        h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1]);
        Gsyn1[i] = g(U1[i]);
        U2[i] = U2[i-1] + dt/C*(-Gm*U2[i-1] + Gsyn1[i-1]*(delEsyn-U2[i-1]) + Gna*mInf(U2[i-1])*h2[i-1]*(delEna-U2[i-1]) + Iapp[i-1]);
        h2[i] = h2[i-1] + dt/tauh(U2[i-1])*(hInf(U2[i-1]) - h2[i-1]);
        Gsyn2[i] = g(U2[i]);

    grid = plt.GridSpec(3,2)
    plt.figure(figsize=(30,15))
    plt.subplot(grid[0,0])
    plt.plot(t,Iapp+Ipert,lw=5)
    plt.ylabel('$I_{app}$ (nA)')

    plt.subplot(grid[1,0])
    plt.plot(t,U1,lw=5,label='Neuron 1')
    plt.plot(t,U2,'--',lw=5,label='Neuron 2')
    plt.ylabel('U (mV)')
    plt.legend()

    plt.subplot(grid[2,0])
    plt.plot(t,h1,lw=5,label='Neuron 1')
    plt.plot(t,h2,'--',lw=5,label='Neuron 2')
    plt.ylabel('h')
    plt.legend()

    plt.subplot(grid[:,1])
    plt.axvline(x=0,linestyle=':',color='k',label='Synaptic Threshold',lw=5)
    plt.xlim([lim[0],lim[1]])
    plt.ylim([lim[2],lim[3]])
    plt.plot(Utest,inUnull(Utest,Iapp[0]),label='U Nullcline for Inhibited Neuron',color='C2',lw=5)
    plt.plot(Utest,hInf(Utest),':',label='h Nullcline',color='C3',lw=5)
    plt.plot(Utest,unUnull(Utest,Iapp[0]),label='U Nullcline for Uninhibited Neuron',color='C4',lw=5)
    plt.plot(U1,h1,label='Neuron 1',color='C0',lw=5)
    plt.plot(U2,h2,'--',label='Neuron 2',color='C1',lw=5)
    plt.xlabel('U (mV)')
    plt.ylabel('h (mV)')
    plt.legend()


In [5]:
doubleNeuronPlot()
doubleNeuronPlot(lim=[-0.05,0.05,0.666,0.668])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
doubleNeuronPlot(delta=0.01)
doubleNeuronPlot(delta=0.01,lim=[-0.05,0.05,0.666,0.668])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
doubleNeuronPlot(delta=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
doubleNeuronPlot(delta=8.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
doubleNeuronPlot(S=0.1)
doubleNeuronPlot(S=0.1,lim=[18,22,0.19,0.23])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Plot the frequency as a function of delta

delta = np.linspace(-2.0,10)
numDelta = np.size(delta)
period = np.empty(numDelta)
#period[:] = np.nan

C = 5 # nF
tauHmax = 300 # ms
Gm = 1 # uS
Ena = 50 # mV

I = 0
Er = -60
k = -1

dt = 1
tMax = 10000
R = 20
t = np.arange(0,tMax,dt)
numSteps = np.size(t)

Iapp = np.zeros(numSteps)
Ipert = np.zeros(numSteps)
Ipert[1] = 1

# Design the synapse
Ein = -100
delEsyn = Ein - R

# persistent sodium channel
delEna = Ena - Er
Utest = np.arange(-R,3*R,0.1)
mInf = lambda U: 1/(1+np.exp(S*(R-U)))
hInf = lambda U: 1/(1+0.5*np.exp(S*U))
tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

# Channel conductance
Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

# h* and m*
Ustar = R
mStar = mInf(Ustar)
hStar = hInf(Ustar)

for d in range(numDelta):


    dUdt = lambda U: -Gm*U + Gna*mInf(U)*hInf(U)*(delEna-U)

    gSyn = (-delta[d] - delta[d]*Gna*mInf(delta[d])*hInf(delta[d]) + Gna*mInf(delta[d])*hInf(delta[d])*delEna)/(delta[d] - delEsyn)

    unUnull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))
    inUnull = lambda U,I: (Gm*U - gSyn*(delEsyn - U) - I)/(Gna*mInf(U)*(delEna - U))

    # Compute simulation
    U1 = np.zeros(numSteps)
    U2 = np.zeros(numSteps)
    h1 = np.zeros(numSteps)
    h1[0] = hInf(U1[0])
    h2 = np.zeros(numSteps)
    h2[0] = hInf(U2[0])
    
    numCrossings = 0
    crossing = np.zeros(2)
    crossing[:] = np.nan
    i = 1
    n = 0

    g = lambda U: min(max(U/R,0),1)*gSyn

    Gsyn1 = np.zeros(numSteps)
    Gsyn1[0] = g(U1[0])
    Gsyn2 = np.zeros(numSteps)
    Gsyn2[0] = g(U2[0])

    while (i < numSteps) and (n < 2):
        U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gsyn2[i-1]*(delEsyn-U1[i-1]) + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1])
        h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1])
        Gsyn1[i] = g(U1[i])
        U2[i] = U2[i-1] + dt/C*(-Gm*U2[i-1] + Gsyn1[i-1]*(delEsyn-U2[i-1]) + Gna*mInf(U2[i-1])*h2[i-1]*(delEna-U2[i-1]) + Iapp[i-1])
        h2[i] = h2[i-1] + dt/tauh(U2[i-1])*(hInf(U2[i-1]) - h2[i-1])
        Gsyn2[i] = g(U2[i])
        if (U1[i-1] < R) and (U1[i]>=R):
            crossing[n] = t[i]
            n += 1
        i += 1
    period[d] = crossing[1] - crossing[0]

plt.figure(figsize=(30,15))
plt.plot(delta,1/period,lw=5)
plt.xlabel('$\delta$')
plt.ylabel('Frequency (Hz)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Frequency (Hz)')

In [18]:
# Phase Response Curve (Direct Current Injection into U1)

deltas = [0.1,1,3]
Istim = [-5,5]
numPhase = 100

#period[:] = np.nan

C = 5 # nF
tauHmax = 300 # ms
Gm = 1 # uS
Ena = 50 # mV

I = 0
Er = -60
k = -1

dt = 1
tMax = 10000
R = 20
t = np.arange(0,tMax,dt)
numSteps = np.size(t)

Iapp = np.zeros(numSteps)


# Design the synapse
Ein = -100
delEsyn = Ein - R

# persistent sodium channel
delEna = Ena - Er
Utest = np.arange(-R,3*R,0.1)
mInf = lambda U: 1/(1+np.exp(S*(R-U)))
hInf = lambda U: 1/(1+0.5*np.exp(S*U))
tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

# Channel conductance
Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

# h* and m*
Ustar = R
mStar = mInf(Ustar)
hStar = hInf(Ustar)

plt.figure(figsize=(30,15))

for stim in range(np.size(Istim)):
    plt.subplot(1,2,stim+1)
    plt.axvline(x=0.5,color='k',linestyle=':')
    plt.xlabel('Phase of Stimulus')
    plt.ylabel('Normalized Phase Advancement')
    plt.xlim([0,1])
    plt.ylim([-1,1])
    plt.title(f'Stimulus = {Istim[stim]} nA')
    for delta in deltas:
        period = np.empty(numPhase+1)


        for p in range(numPhase):
            Ipert = np.zeros(numSteps)
            Ipert[0] = 1
            if p > 0:
                Ipert[round(tStim[p-1]/dt):round((tStim[p-1]+0.05*period[0])/dt)] = Istim[stim]

            dUdt = lambda U: -Gm*U + Gna*mInf(U)*hInf(U)*(delEna-U)

            gSyn = (-delta - delta*Gna*mInf(delta)*hInf(delta) + Gna*mInf(delta)*hInf(delta)*delEna)/(delta - delEsyn)

            unUnull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))
            inUnull = lambda U,I: (Gm*U - gSyn*(delEsyn - U) - I)/(Gna*mInf(U)*(delEna - U))

            # Compute simulation
            U1 = np.zeros(numSteps)
            U2 = np.zeros(numSteps)
            h1 = np.zeros(numSteps)
            h1[0] = hInf(U1[0])
            h2 = np.zeros(numSteps)
            h2[0] = hInf(U2[0])

            numCrossings = 0
            crossing = np.zeros(2)
            crossing[:] = np.nan
            i = 1
            n = 0

            g = lambda U: min(max(U/R,0),1)*gSyn

            Gsyn1 = np.zeros(numSteps)
            Gsyn1[0] = g(U1[0])
            Gsyn2 = np.zeros(numSteps)
            Gsyn2[0] = g(U2[0])

            while (i < numSteps) and (n < 2):
                U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gsyn2[i-1]*(delEsyn-U1[i-1]) + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1])
                h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1])
                Gsyn1[i] = g(U1[i])
                U2[i] = U2[i-1] + dt/C*(-Gm*U2[i-1] + Gsyn1[i-1]*(delEsyn-U2[i-1]) + Gna*mInf(U2[i-1])*h2[i-1]*(delEna-U2[i-1]) + Iapp[i-1])
                h2[i] = h2[i-1] + dt/tauh(U2[i-1])*(hInf(U2[i-1]) - h2[i-1])
                Gsyn2[i] = g(U2[i])
                if (U1[i-1] < R) and (U1[i]>=R):
                    crossing[n] = t[i]
                    n += 1
                i += 1
            period[p] = crossing[1] - crossing[0]
            if p==0:
                tStim = crossing[0] + np.arange(numPhase)/numPhase*period[0]
                #print(tStim)

            #plt.plot(t,U1,label=str(p))
        #plt.legend(loc=7)    
        
        plt.plot(np.arange(numPhase-1)/numPhase,-(period[1:-1]-period[0])/period[0],lw=5,label=str(delta))
    plt.legend()
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Phase Response Curve (Synaptic Current Injection into U1)

delta = 0.1
#Istim = [-5,5]
numPhase = 100
gStim = 0.05
eStim = [0,20]

#period[:] = np.nan

C = 5 # nF
tauHmax = 300 # ms
Gm = 1 # uS
Ena = 50 # mV

I = 0
Er = -60
k = -1

dt = 1
tMax = 10000
R = 20
t = np.arange(0,tMax,dt)
numSteps = np.size(t)

Iapp = np.zeros(numSteps)
Ipert = np.zeros(numSteps)
Ipert[1] = 1

# Design the synapse
Ein = -100
delEsyn = Ein - R

# persistent sodium channel
delEna = Ena - Er
Utest = np.arange(-R,3*R,0.1)
mInf = lambda U: 1/(1+np.exp(S*(R-U)))
hInf = lambda U: 1/(1+0.5*np.exp(S*U))
tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

# Channel conductance
Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

# h* and m*
Ustar = R
mStar = mInf(Ustar)
hStar = hInf(Ustar)

plt.figure(figsize=(30,15))

for stim in range(np.size(eStim)):
    period = np.empty(numPhase+1)
    for p in range(numPhase):
        gPert = np.zeros(numSteps)
        
        if p > 0:
            #iStim = round(tStim[p-1]/dt)
            #Ipert[round(tStim[p-1]/dt):round(tStim[p]/dt)] = Istim[stim]
            gPert[round(tStim[p-1]/dt):round((tStim[p-1]+0.05*period[0])/dt)] = gStim

        dUdt = lambda U: -Gm*U + Gna*mInf(U)*hInf(U)*(delEna-U)

        gSyn = (-delta - delta*Gna*mInf(delta)*hInf(delta) + Gna*mInf(delta)*hInf(delta)*delEna)/(delta - delEsyn)

        unUnull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))
        inUnull = lambda U,I: (Gm*U - gSyn*(delEsyn - U) - I)/(Gna*mInf(U)*(delEna - U))

        # Compute simulation
        U1 = np.zeros(numSteps)
        U2 = np.zeros(numSteps)
        h1 = np.zeros(numSteps)
        h1[0] = hInf(U1[0])
        h2 = np.zeros(numSteps)
        h2[0] = hInf(U2[0])

        numCrossings = 0
        crossing = np.zeros(2)
        crossing[:] = np.nan
        i = 1
        n = 0

        g = lambda U: min(max(U/R,0),1)*gSyn

        Gsyn1 = np.zeros(numSteps)
        Gsyn1[0] = g(U1[0])
        Gsyn2 = np.zeros(numSteps)
        Gsyn2[0] = g(U2[0])

        while (i < numSteps) and (n < 2):
            U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gsyn2[i-1]*(delEsyn-U1[i-1]) + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1]+gPert[i-1]*(eStim[stim]-U1[i-1]))
            h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1])
            Gsyn1[i] = g(U1[i])
            U2[i] = U2[i-1] + dt/C*(-Gm*U2[i-1] + Gsyn1[i-1]*(delEsyn-U2[i-1]) + Gna*mInf(U2[i-1])*h2[i-1]*(delEna-U2[i-1]) + Iapp[i-1])
            h2[i] = h2[i-1] + dt/tauh(U2[i-1])*(hInf(U2[i-1]) - h2[i-1])
            Gsyn2[i] = g(U2[i])
            if (U1[i-1] < R) and (U1[i]>=R):
                crossing[n] = t[i]
                n += 1
            i += 1
        period[p] = crossing[1] - crossing[0]
        if p==0:
            tStim = crossing[0] + np.arange(numPhase)/numPhase*period[0]
            
    plt.subplot(1,2,stim+1)
    plt.axvline(x=0.5,color='k',linestyle=':')
    plt.plot(np.arange(numPhase-1)/numPhase,-(period[1:-1]-period[0])/period[0],lw=5)
    plt.xlabel('Phase of Stimulus')
    plt.ylabel('$\Delta$ Phase')
    plt.xlim([0,1])
    plt.ylim([-1,1])
    #title = str('$I_{stim}$ = ',str())
    if eStim[stim] > 0:
        plt.title('Excitatory Stimulus')
    else:
        plt.title('Inhibitory Stimulus')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Make a video
def singleNeuronVideo(name):
    # Parameters
    C = 5 # nF
    tauHmax = 300 # ms
    Gm = 1 # uS
    Ena = 50 # mV

    I = 0
    tStart = 1000
    tEnd = 4000
    Er = -60
    k = -1
    tTail = 2500

    
    delta = -0.01
    dt = 1
    tMax = 500
    R = 20
    t = np.arange(0,tMax,dt)
    numSteps = np.size(t)

    Iapp = np.zeros(numSteps)
    for i in range(numSteps):
        if (t[i]>= tStart & t[i]<=tEnd):
            Iapp[i] = I

    Ipert = np.zeros(numSteps)
    Ipert[1] = 1

    # Design the synapse
    Ein = -100
    delEsyn = Ein - R

    # persistent sodium channel
    S = 0.05
    delEna = Ena - Er
    Utest = np.arange(-R,3*R,0.1)
    mInf = lambda U: 1/(1+np.exp(S*(R-U)))
    hInf = lambda U: 1/(1+0.5*np.exp(S*U))
    tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

    # Channel conductance
    Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

    # h* and m*
    Ustar = R
    mStar = mInf(Ustar)
    hStar = hInf(Ustar)

    Unull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))

    # Compute simulation
    U1 = np.zeros(numSteps)
    h1 = np.zeros(numSteps)
    h1[0] = hInf(U1[0])

    g = lambda U: min(max(U/R,0),1)*gSyn


    for i in range(1,numSteps):
        U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1]);
        h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1]);

    grid = plt.GridSpec(3,2)
    fig = plt.figure(figsize=(30,15))
    axIapp = plt.subplot(grid[0,0])
    axU = plt.subplot(grid[1,0])
    axh = plt.subplot(grid[2,0])
    axPhase =  plt.subplot(grid[:,1])
    
    axIapp.set(xlim=(min(t),max(t)),ylim=(0,2),xlabel='Time (ms)',ylabel='$I_{app}$ (nA)')
    cIapp, = axIapp.plot([],[],lw=5)

    axU.set(xlim=(min(t),max(t)),ylim=(-R,3*R),xlabel='Time (ms)',ylabel='U (mV)')
    cU, = axU.plot([],[],lw=5)
    
    axh.set(xlim=(min(t),max(t)),ylim=(0,1),xlabel='Time (ms)',ylabel='h (mV)')
    ch, = axh.plot([],[],lw=5)

    axPhase.set(xlim=(-R,3*R),ylim=(0,1),xlabel='U (mV)',ylabel='h (mV)')
    axPhase.axvline(x=0,linestyle=':',color='k',label='Synaptic Threshold',lw=5)
    chn, = axPhase.plot(Utest,hInf(Utest),':',label='h Nullcline',lw=5,color='C2')
    cUn, = axPhase.plot(Utest,Unull(Utest,Iapp[0]),label='U Nullcline',lw=5,color='C3')
    cUp, = axPhase.plot([],[],label='Neuron',lw=5,color='C0')
    cMark, = axPhase.plot([],[],'o',markersize=20,color='C0')
    axPhase.legend()
    
    iTail = round(tTail,dt)
    
    def drawFrame(i):
        print(numSteps-i)
        cIapp.set_data(t[:i],Iapp[:i])
        cU.set_data(t[:i],U1[:i])
        ch.set_data(t[:i],h1[:i])
        cUp.set_data(U1[max(1,i-iTail):i],h1[max(1,i-iTail):i])
        cMark.set_data(U1[i],h1[i])
        
        return(cIapp,cU,ch,cUp,cMark)
    
    writervideo = animation.FFMpegWriter(fps=60)
    anim = animation.FuncAnimation(fig, drawFrame, frames=numSteps,interval=0.5, blit=True,repeat=False,save_count=numSteps)
    anim.save(name,writer=writervideo)

In [14]:
# Make a video
def doubleNeuronVideo(name,delta=-0.01,S=0.05,lim=[-R,3*R,0,1]):
    # Parameters
    C = 5 # nF
    tauHmax = 300 # ms
    Gm = 1 # uS
    Ena = 50 # mV

    I = 0
    tStart = 1000
    tEnd = 4000
    Er = -60
    k = -1
    tTail = 2500

    dt = 1
    tMax = 5000
    R = 20
    t = np.arange(0,tMax,dt)
    numSteps = np.size(t)

    Iapp = np.zeros(numSteps)
    for i in range(numSteps):
        if (t[i]>= tStart & t[i]<=tEnd):
            Iapp[i] = I

    Ipert = np.zeros(numSteps)
    Ipert[1] = 1

    # Design the synapse
    Ein = -100
    delEsyn = Ein - R

    # persistent sodium channel
    delEna = Ena - Er
    Utest = np.arange(-R,3*R,0.1)
    mInf = lambda U: 1/(1+np.exp(S*(R-U)))
    hInf = lambda U: 1/(1+0.5*np.exp(S*U))
    tauh = lambda U: tauHmax*hInf(U)*np.sqrt(0.5*np.exp(S*U))

    # Channel conductance
    Gna = Gm*R/(mInf(R)*hInf(R)*(delEna-R))

    # h* and m*
    Ustar = R
    mStar = mInf(Ustar)
    hStar = hInf(Ustar)

    # compute jacobian matrix and its eigenvalues
    dU = 1e-3
    dm_dU = lambda U: (mInf(U+dU)-mInf(U))/dU
    dh_dU = lambda U: (hInf(U+dU)-hInf(U))/dU
    dtauh_dU = lambda U: (tauh(U+dU)-tauh(U))/dU

    dUdot_dU = 1/C*(-1+dm_dU(Ustar)*Gna*hStar*(delEna - Ustar) - Gna*mStar*hStar)
    dUdot_dh = 1/C*(Gna*mStar*(delEna - Ustar))
    dhdot_dU = dh_dU(Ustar)/tauh(Ustar)
    dhdot_dh = -1/tauh(Ustar)

    J = np.array([[dUdot_dU, dUdot_dh],
                   [dhdot_dU, dhdot_dh]])

    eigVals,_ = np.linalg.eig(J)
    if np.imag(J).any():
        print('Warning: Complex Eigenvalues!')

    dUdt = lambda U: -Gm*U + Gna*mInf(U)*hInf(U)*(delEna-U)

    gSyn = (-delta - delta*Gna*mInf(delta)*hInf(delta) + Gna*mInf(delta)*hInf(delta)*delEna)/(delta - delEsyn)
   
    unUnull = lambda U,I: (Gm*U - I)/(Gna*mInf(U)*(delEna - U))
    inUnull = lambda U,I: (Gm*U - gSyn*(delEsyn - U) - I)/(Gna*mInf(U)*(delEna - U))

    # Compute simulation
    U1 = np.zeros(numSteps)
    U2 = np.zeros(numSteps)
    h1 = np.zeros(numSteps)
    h1[0] = hInf(U1[0])
    h2 = np.zeros(numSteps)
    h2[0] = hInf(U2[0])

    g = lambda U: min(max(U/R,0),1)*gSyn

    Gsyn1 = np.zeros(numSteps)
    Gsyn1[0] = g(U1[0])
    Gsyn2 = np.zeros(numSteps)
    Gsyn2[0] = g(U2[0])

    for i in range(1,numSteps):
        U1[i] = U1[i-1] + dt/C*(-Gm*U1[i-1] + Gsyn2[i-1]*(delEsyn-U1[i-1]) + Gna*mInf(U1[i-1])*h1[i-1]*(delEna-U1[i-1]) + Iapp[i-1] + Ipert[i-1]);
        h1[i] = h1[i-1] + dt/tauh(U1[i-1])*(hInf(U1[i-1]) - h1[i-1]);
        Gsyn1[i] = g(U1[i]);
        U2[i] = U2[i-1] + dt/C*(-Gm*U2[i-1] + Gsyn1[i-1]*(delEsyn-U2[i-1]) + Gna*mInf(U2[i-1])*h2[i-1]*(delEna-U2[i-1]) + Iapp[i-1]);
        h2[i] = h2[i-1] + dt/tauh(U2[i-1])*(hInf(U2[i-1]) - h2[i-1]);
        Gsyn2[i] = g(U2[i]);
    
    grid = plt.GridSpec(3,2)
    fig = plt.figure(figsize=(30,15))
    axIapp = plt.subplot(grid[0,0])
    axU = plt.subplot(grid[1,0])
    axh = plt.subplot(grid[2,0])
    axPhase =  plt.subplot(grid[:,1])
    
    axIapp.set(xlim=(min(t),max(t)),ylim=(0,2),xlabel='Time (ms)',ylabel='$I_{app}$ (nA)')
    cIapp, = axIapp.plot([],[],lw=5)

    axU.set(xlim=(min(t),max(t)),ylim=(-R,3*R),xlabel='Time (ms)',ylabel='U (mV)')
    cU1, = axU.plot([],[],lw=5,label='$U_1$')
    cU2, = axU.plot([],[],lw=5,label='$U_2$')
    axU.legend()
    
    axh.set(xlim=(min(t),max(t)),ylim=(0,1),xlabel='Time (ms)',ylabel='h (mV)')
    ch1, = axh.plot([],[],lw=5,label='$h_1$')
    ch2, = axh.plot([],[],lw=5,label='$h_2$')
    axh.legend()

    axPhase.set(xlim=(lim[0],lim[1]),ylim=(lim[2],lim[3]),xlabel='U (mV)',ylabel='h (mV)')
    axPhase.axvline(x=0,linestyle=':',color='k',label='Synaptic Threshold',lw=5)
    axPhase.plot(Utest,hInf(Utest),':',label='h Nullcline',lw=5,color='C2')
    axPhase.plot(Utest,inUnull(Utest,Iapp[0]),label='U Nullcline for Inhibited Neuron',lw=5,color='C3')
    axPhase.plot(Utest,unUnull(Utest,Iapp[0]),label='U Nullcline for Uninhibited Neuron',lw=5,color='C4')
    cU2p, = axPhase.plot([],[],label='Neuron 2',lw=5,color='C1')
    cMark2, = axPhase.plot([],[],'o',markersize=20,color='C1')
    cU1p, = axPhase.plot([],[],label='Neuron 1',lw=5,color='C0')
    cMark1, = axPhase.plot([],[],'o',markersize=20,color='C0')
    axPhase.legend()
    
    iTail = round(tTail,dt)
    
    def drawFrame(i):
        print(numSteps-i)
        cIapp.set_data(t[:i],Iapp[:i])
        cU1.set_data(t[:i],U1[:i])
        ch1.set_data(t[:i],h1[:i])
        cU1p.set_data(U1[max(1,i-iTail):i],h1[max(1,i-iTail):i])
        cMark1.set_data(U1[i],h1[i])
        cU2.set_data(t[:i],U2[:i])
        ch2.set_data(t[:i],h2[:i])
        cU2p.set_data(U2[max(1,i-iTail):i],h2[max(1,i-iTail):i])
        cMark2.set_data(U2[i],h2[i])
        
        return(cIapp,cU1,ch1,cU1p,cMark1,cU2,ch2,cU2p,cMark2)
    
    writervideo = animation.FFMpegWriter(fps=60)
    anim = animation.FuncAnimation(fig, drawFrame, frames=numSteps,interval=0.5, blit=True,repeat=False,save_count=numSteps)
    anim.save(name,writer=writervideo)